<a href="https://colab.research.google.com/github/belovelace/Once-Upon-A-Time-AI/blob/main/TTSv3_%ED%95%99%EC%8A%B5_(V_4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎙️ 한국 전래동화 오디오북 생성 시스템 (GPT-SoVITS v3)

**목적**: GPT-SoVITS v3 모델을 활용한 한국어 캐릭터 음성 기반 동화 오디오북 생성

**주요 기능**:
- 한국어 TTS 지원 (v3 모델)
- 12개 캐릭터 음성 (AI Hub 데이터)
- JSON 스크립트 기반 오디오북 자동 생성
- 실시간 음성 합성

## 📋 STEP 0: 환경 확인 및 구글 드라이브 연결

In [ ]:
import torch
import os

print("="*60)
print("환경 확인")
print("="*60)
print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ CUDA 버전: {torch.version.cuda}")

# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ 구글 드라이브 연결 완료!")

환경 확인
✓ PyTorch: 2.9.0+cu126
✓ CUDA 사용 가능: True
✓ GPU: Tesla T4
✓ CUDA 버전: 12.6
Mounted at /content/drive

✅ 구글 드라이브 연결 완료!


## 📂 STEP 0.5: 참조 오디오 데이터 준비

**중요**: 오디오북 생성을 위해 참조 오디오 파일이 필요합니다.

두 가지 옵션:
1. **전체 파이프라인** (아래 셀 실행): 구글 드라이브의 12개 캐릭터 음성 데이터를 준비
2. **간단한 테스트** (셀 건너뛰기): STEP 6에서 임시 참조 오디오 사용

In [ ]:
# 🎭 한국 전래동화 캐릭터 음성 데이터 준비 (구글 드라이브 버전)
# Google Colab에서 실행

import os
import shutil
from pathlib import Path
import json

# ========================================
# STEP 0: 구글 드라이브 마운트
# ========================================

print("📂 구글 드라이브 연결 중...")

from google.colab import drive
drive.mount('/content/drive')

print("✅ 구글 드라이브 연결 완료!")


📂 구글 드라이브 연결 중...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 구글 드라이브 연결 완료!


In [ ]:

# ========================================
# STEP 1: 소스 폴더 경로 설정
# ========================================

print("\n" + "="*60)
print("STEP 1: 데이터 위치 확인")
print("="*60)

# 구글 드라이브의 audio_sample 폴더 경로
# 실제 경로에 맞게 수정하세요
source_dir = "/content/drive/MyDrive/P_dev/audio_sample"

# 경로 확인
if not os.path.exists(source_dir):
    print(f"\n⚠️ 폴더를 찾을 수 없습니다: {source_dir}")
    print("\n구글 드라이브 구조를 확인하세요:")
    print("  1. 왼쪽 사이드바 '파일' 아이콘 클릭")
    print("  2. drive > MyDrive 하위 경로 확인")
    print("  3. 위 source_dir 경로를 실제 경로로 수정")
    raise FileNotFoundError(f"폴더 없음: {source_dir}")

print(f"\n✅ 소스 폴더 발견: {source_dir}")

# 폴더 목록 확인
voice_folders = sorted([f for f in os.listdir(source_dir)
                       if os.path.isdir(os.path.join(source_dir, f))])

print(f"\n📋 발견된 성우 폴더 ({len(voice_folders)}개):")
for folder in voice_folders:
    folder_path = os.path.join(source_dir, folder)
    wav_count = len(list(Path(folder_path).glob("*.wav")))
    print(f"  ✓ {folder}: {wav_count}개 WAV 파일")

# ========================================
# STEP 2: 캐릭터 매핑 정의 (12개)
# ========================================

print("\n" + "="*60)
print("STEP 2: 캐릭터 매핑 설정")
print("="*60)

# 전체 12개 캐릭터 매핑
character_mapping = {
    # 남성 캐릭터
    "boy": "M0101_명량한 소년",
    "scholar": "M0202_선비",
    "nolbu": "M0303_놀부",
    "king": "M0404_왕",
    "commander": "M0505_지휘관",
    "bachelor": "M0606_총각",

    # 여성 캐릭터
    "teacher": "W0107_선생님",
    "librarian": "W0208_사서",
    "queen": "W0309_여왕",
    "girl": "W0410_소녀",
    "grandma": "W0511_할머니",
    "princess": "W0612_공주"
}

print("\n🎭 캐릭터 목록:")
print("\n[남성 캐릭터]")
male_chars = ["boy", "scholar", "nolbu", "king", "commander", "bachelor"]
for char in male_chars:
    print(f"  • {char:12s} ← {character_mapping[char]}")

print("\n[여성 캐릭터]")
female_chars = ["teacher", "librarian", "queen", "girl", "grandma", "princess"]
for char in female_chars:
    print(f"  • {char:12s} ← {character_mapping[char]}")

# ========================================
# STEP 3: GPT-SoVITS용 데이터셋 구성
# ========================================

print("\n" + "="*60)
print("STEP 3: 데이터셋 재구성")
print("="*60)

output_base = "/content/GPT-SoVITS/dataset"
os.makedirs(output_base, exist_ok=True)

print("\n📁 파일 복사 중...")

for character, folder_name in character_mapping.items():
    src_folder = os.path.join(source_dir, folder_name)
    dst_folder = os.path.join(output_base, character)

    # 폴더 없으면 건너뛰기
    if not os.path.exists(src_folder):
        print(f"  ⚠️ {character}: 소스 폴더 없음 ({folder_name})")
        continue

    os.makedirs(dst_folder, exist_ok=True)

    # WAV 파일 복사 및 이름 변경
    wav_files = list(Path(src_folder).glob("*.wav"))

    for i, wav_file in enumerate(wav_files):
        # 통일된 파일명: character_0001.wav
        new_name = f"{character}_{i:04d}.wav"
        dst_path = os.path.join(dst_folder, new_name)

        shutil.copy2(str(wav_file), dst_path)

    print(f"  ✅ {character:12s}: {len(wav_files):3d}개 파일")

print("\n✅ 데이터셋 구성 완료!")

# ========================================
# STEP 4: 데이터 통계 확인
# ========================================

print("\n" + "="*60)
print("STEP 4: 데이터 품질 검사")
print("="*60)

# librosa 설치
print("\n📦 필요한 라이브러리 설치 중...")
!pip install librosa soundfile -q

import librosa
import numpy as np

print("\n📊 캐릭터별 음성 데이터 통계:\n")

total_minutes = 0
stats_list = []

for character in character_mapping.keys():
    char_dir = os.path.join(output_base, character)

    if not os.path.exists(char_dir):
        continue

    wav_files = list(Path(char_dir).glob("*.wav"))

    if not wav_files:
        print(f"  ⚠️ {character:12s}: 파일 없음")
        continue

    # 샘플 10개로 평균 계산 (시간 절약)
    sample_files = wav_files[:min(10, len(wav_files))]
    durations = []
    sample_rates = []

    for wav_file in sample_files:
        try:
            y, sr = librosa.load(str(wav_file), sr=None)
            duration = librosa.get_duration(y=y, sr=sr)
            durations.append(duration)
            sample_rates.append(sr)
        except Exception as e:
            print(f"    ⚠️ 파일 로드 실패: {wav_file.name}")

    if durations:
        avg_duration = np.mean(durations)
        estimated_total = avg_duration * len(wav_files)
        minutes = estimated_total / 60
        total_minutes += minutes

        # 통계 저장
        stats_list.append({
            'character': character,
            'files': len(wav_files),
            'minutes': minutes,
            'avg_sr': int(np.mean(sample_rates))
        })

        # 상태 표시
        if minutes >= 10:
            status = "✅ 우수"
        elif minutes >= 5:
            status = "✅ 양호"
        else:
            status = "⚠️ 부족"

        print(f"  {status} {character:12s}: {minutes:5.1f}분 ({len(wav_files):3d}개, {int(np.mean(sample_rates))}Hz)")

print(f"\n📈 전체 통계:")
print(f"  • 총 캐릭터: {len(stats_list)}개")
print(f"  • 총 음성 길이: {total_minutes:.1f}분 ({total_minutes/60:.1f}시간)")
print(f"  • 캐릭터당 평균: {total_minutes/len(stats_list):.1f}분")

# 권장사항
insufficient = [s for s in stats_list if s['minutes'] < 5]
if insufficient:
    print(f"\n⚠️ 5분 미만 캐릭터 ({len(insufficient)}개):")
    for s in insufficient:
        print(f"    • {s['character']}: {s['minutes']:.1f}분")
    print("  → 추가 음성 데이터 수집 권장")

# ========================================
# STEP 5: 참조 오디오 정보 생성
# ========================================

print("\n" + "="*60)
print("STEP 5: 참조 오디오 설정")
print("="*60)

# 각 캐릭터의 대표 참조 텍스트
reference_texts = {
    "boy": "나는 씩씩한 소년이야!",
    "scholar": "학문의 길은 멀고도 험하도다.",
    "nolbu": "이놈, 감히 형님한테 대들어!",
    "king": "짐이 명하노니, 모두 물러가거라.",
    "commander": "전군, 집결하라!",
    "bachelor": "저는 총각입니다.",
    "teacher": "자, 이제 공부를 시작해볼까요?",
    "librarian": "조용히 하세요. 여기는 도서관입니다.",
    "queen": "나는 이 나라의 여왕이니라.",
    "girl": "엄마, 나 배고파!",
    "grandma": "옛날 옛날에 말이야.",
    "princess": "저는 아름다운 공주랍니다."
}

reference_info = {}

print("\n🎯 각 캐릭터의 참조 오디오 선정:\n")

for character in character_mapping.keys():
    char_dir = os.path.join(output_base, character)

    if not os.path.exists(char_dir):
        continue

    wav_files = sorted(list(Path(char_dir).glob("*.wav")))

    if wav_files:
        # 첫 번째 파일을 참조로 사용
        ref_audio = str(wav_files[0])

        reference_info[character] = {
            "ref_audio_path": ref_audio,
            "ref_text": reference_texts.get(character, "안녕하세요"),
            "voice_type": character_mapping[character],
            "total_files": len(wav_files)
        }

        print(f"  {character:12s}: {wav_files[0].name}")

# JSON 저장
ref_json_path = "/content/GPT-SoVITS/character_references.json"
with open(ref_json_path, 'w', encoding='utf-8') as f:
    json.dump(reference_info, f, ensure_ascii=False, indent=2)

print(f"\n✅ 참조 정보 저장: {ref_json_path}")

# ========================================
# STEP 6: 슬라이싱 생략 - 복사
# ========================================

print("\n" + "="*60)
print("STEP 6: 슬라이싱 단계 생략")
print("="*60)

sliced_base = "/content/GPT-SoVITS/sliced"

print("\n🔄 dataset → sliced 복사 중...")

if os.path.exists(output_base):
    shutil.copytree(output_base, sliced_base, dirs_exist_ok=True)
    print("✅ 복사 완료!")
    print(f"  경로: {sliced_base}")

# ========================================
# STEP 7: 완료 및 다음 단계 안내
# ========================================

print("\n" + "="*60)
print("✅ 모든 데이터 준비 완료!")
print("="*60)

print("\n📂 생성된 폴더 구조:")
print("  /content/GPT-SoVITS/")
print("    ├── dataset/        (원본)")
print("    │   ├── boy/")
print("    │   ├── scholar/")
print("    │   ├── nolbu/")
print("    │   └── ... (12개)")
print("    └── sliced/         (학습용)")
print("        ├── boy/")
print("        └── ... (12개)")

print("\n📄 생성된 파일:")
print(f"  • {ref_json_path}")

print("\n" + "="*60)
print("🎯 다음 단계: ASR 실행")
print("="*60)

print("\n아래 코드를 복사해서 새 셀에서 실행하세요:\n")
print("-" * 60)
print("""
# ASR (음성 → 텍스트 자동 생성)
characters = [
    "boy", "scholar", "nolbu", "king", "commander", "bachelor",
    "teacher", "librarian", "queen", "girl", "grandma", "princess"
]

for char in characters:
    print(f'\\n🗣️ {char} ASR 진행 중...')

    !cd /content/GPT-SoVITS && python tools/asr/funasr_asr.py \\
        -i /content/GPT-SoVITS/sliced/{char} \\
        -o /content/GPT-SoVITS/asr/{char} \\
        -s large \\
        -l ko

    print(f'✅ {char} ASR 완료')

print('\\n✅ 모든 캐릭터 ASR 완료!')
""")
print("-" * 60)

# 추가 정보 출력
print("\n" + "="*60)
print("📌 중요 정보")
print("="*60)
print(f"\n• 총 캐릭터 수: {len(character_mapping)}개")
print(f"• 총 음성 길이: {total_minutes:.1f}분")
print(f"• ASR 예상 소요 시간: {len(character_mapping) * 5}~{len(character_mapping) * 10}분")

print("\n💡 선택적 학습 권장:")
print("  12개 모두 학습하면 15-20시간 소요")
print("  주요 캐릭터 7개만 선택 권장:")
print("  → princess, boy, grandma, king, girl, scholar, nolbu")

print("\n✨ 준비 완료! ASR을 실행하세요.")


STEP 1: 데이터 위치 확인

✅ 소스 폴더 발견: /content/drive/MyDrive/P_dev/audio_sample

📋 발견된 성우 폴더 (12개):
  ✓ M0101_명량한 소년: 7개 WAV 파일
  ✓ M0202_선비: 8개 WAV 파일
  ✓ M0303_놀부: 5개 WAV 파일
  ✓ M0404_왕: 4개 WAV 파일
  ✓ M0505_지휘관: 10개 WAV 파일
  ✓ M0606_총각: 7개 WAV 파일
  ✓ W0107_선생님: 7개 WAV 파일
  ✓ W0208_사서: 14개 WAV 파일
  ✓ W0309_여왕: 4개 WAV 파일
  ✓ W0410_소녀: 13개 WAV 파일
  ✓ W0511_할머니: 5개 WAV 파일
  ✓ W0612_공주: 8개 WAV 파일

STEP 2: 캐릭터 매핑 설정

🎭 캐릭터 목록:

[남성 캐릭터]
  • boy          ← M0101_명량한 소년
  • scholar      ← M0202_선비
  • nolbu        ← M0303_놀부
  • king         ← M0404_왕
  • commander    ← M0505_지휘관
  • bachelor     ← M0606_총각

[여성 캐릭터]
  • teacher      ← W0107_선생님
  • librarian    ← W0208_사서
  • queen        ← W0309_여왕
  • girl         ← W0410_소녀
  • grandma      ← W0511_할머니
  • princess     ← W0612_공주

STEP 3: 데이터셋 재구성

📁 파일 복사 중...
  ✅ boy         :   7개 파일
  ✅ scholar     :   8개 파일
  ✅ nolbu       :   5개 파일
  ✅ king        :   4개 파일
  ✅ commander   :  10개 파일
  ✅ bachel

## 📥 STEP 1: GPT-SoVITS 설치

In [ ]:
# GPT-SoVITS 저장소 클론
!git clone https://github.com/RVC-Boss/GPT-SoVITS.git /content/GPT-SoVITS
%cd /content/GPT-SoVITS

print("✅ GPT-SoVITS 클론 완료!")

Cloning into '/content/GPT-SoVITS'...
remote: Enumerating objects: 5780, done.
remote: Total 5780 (delta 0), reused 0 (delta 0), pack-reused 5780 (from 1)
Receiving objects: 100% (5780/5780), 14.01 MiB | 12.00 MiB/s, done.
Resolving deltas: 100% (3322/3322), done.
/content/GPT-SoVITS
✅ GPT-SoVITS 클론 완료!


## 📦 STEP 2: 의존성 패키지 설치

In [ ]:
# ⚠️ 중요: numpy 먼저 설치 (바이너리 호환성 문제 해결)
!pip uninstall -y numpy scipy
!pip install numpy==1.26.4

# PyTorch 2.5.1 설치 (CUDA 12.1)
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# scipy를 numpy 설치 후 재설치
!pip install scipy

# 기본 의존성
!pip install -q -r requirements.txt

# 추가 필수 패키지
!pip install -q cn2an pypinyin funasr==1.0.27
!pip install -q faster-whisper
!pip install -q ffmpeg-python
!pip install -q torchcodec

# transformers와 peft 호환 버전 설치
!pip install -q transformers==4.48.0 peft==0.13.2

# ffmpeg 시스템 설치
!apt-get install -y -qq ffmpeg

print("\n✅ 모든 패키지 설치 완료!")

# 설치 확인
import numpy as np
print(f"✓ NumPy 버전: {np.__version__}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 95.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 107.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━

## 🤖 STEP 3: 사전 학습 모델 다운로드

In [ ]:
# 1단계: numpy와 scipy 완전 삭제
!pip uninstall -y numpy scipy

# 2단계: numpy 1.26.4 설치
!pip install numpy==1.26.4

# 3단계: scipy 재설치
!pip install scipy

# 4단계: 런타임 재시작 필수!
import os
os.kill(os.getpid(), 9)  # 자동 재시작

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: scipy 1.16.3
Uninstalling scipy-1.16.3:
  Successfully uninstalled scipy-1.16.3
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.10.2 requires scipy>=1.2.0, which is not installed.
funasr 1.0.27 requires scipy>=1.4.1, which is not installed.
statsmodels 0.14.5 requires scipy!=1.9.2,>=1.8, which is not installed.
xarray-einstats 0.9.1 requires scipy>=1.11, which is not installed.
imbalanced-learn 0.14.0 requires scipy<2,>=1.11.4, which is not installed.
mizani 0.13.5 requires scipy>=1.8.0, which is not installed.
spopt 0.7

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
^C


In [ ]:
import os
os.chdir('/content/GPT-SoVITS')

# Git LFS 설치
!apt-get install -y -qq git-lfs
!git lfs install

print("📥 v3 한국어 지원 모델 다운로드 중...")

# v3 모델 (한국어 지원)
!mkdir -p GPT_SoVITS/pretrained_models
!wget -q https://huggingface.co/lj1995/GPT-SoVITS/resolve/main/s1v3.ckpt -P GPT_SoVITS/pretrained_models/
!wget -q https://huggingface.co/lj1995/GPT-SoVITS/resolve/main/s2Gv3.pth -P GPT_SoVITS/pretrained_models/

# BigVGAN vocoder
!git clone -q https://huggingface.co/nvidia/bigvgan_v2_24khz_100band_256x \
    GPT_SoVITS/pretrained_models/models--nvidia--bigvgan_v2_24khz_100band_256x

# BERT 모델
!GIT_LFS_SKIP_SMUDGE=0 git clone -q https://huggingface.co/hfl/chinese-roberta-wwm-ext-large \
    GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large

# Hubert 모델
!GIT_LFS_SKIP_SMUDGE=0 git clone -q https://huggingface.co/TencentGameMate/chinese-hubert-base \
    GPT_SoVITS/pretrained_models/chinese-hubert-base

print("\n✅ 모든 모델 다운로드 완료!")
!ls -lh GPT_SoVITS/pretrained_models/

Updated git hooks.
Git LFS initialized.
📥 v3 한국어 지원 모델 다운로드 중...
fatal: destination path 'GPT_SoVITS/pretrained_models/models--nvidia--bigvgan_v2_24khz_100band_256x' already exists and is not an empty directory.
fatal: destination path 'GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large' already exists and is not an empty directory.
fatal: destination path 'GPT_SoVITS/pretrained_models/chinese-hubert-base' already exists and is not an empty directory.

✅ 모든 모델 다운로드 완료!
total 1.8G
drwxr-xr-x 3 root root 4.0K Dec 11 10:39 chinese-hubert-base
drwxr-xr-x 3 root root 4.0K Dec 11 10:38 chinese-roberta-wwm-ext-large
drwxr-xr-x 5 root root 4.0K Dec 11 10:39 models--nvidia--bigvgan_v2_24khz_100band_256x
-rw-r--r-- 1 root root 149M Dec 11 10:36 s1v3.ckpt
-rw-r--r-- 1 root root 149M Dec 11 10:41 s1v3.ckpt.1
-rw-r--r-- 1 root root 734M Dec 11 10:36 s2Gv3.pth
-rw-r--r-- 1 root root 734M Dec 11 10:41 s2Gv3.pth.1


## 🔧 STEP 4: 코드 패치 (BigVGAN import 경로 수정)

In [ ]:
import os
import re

print("🔧 BigVGAN import 경로 수정 중...")

bigvgan_dir = "/content/GPT-SoVITS/GPT_SoVITS/pretrained_models/models--nvidia--bigvgan_v2_24khz_100band_256x"

# 1. __init__.py 패치
init_file = os.path.join(bigvgan_dir, "__init__.py")
if not os.path.exists(init_file):
    with open(init_file, 'w', encoding='utf-8') as f:
        f.write('# BigVGAN module\n')
    print(f"✅ {init_file} 생성됨")

# 2. meldataset.py에서 import 경로 수정
mel_file = os.path.join(bigvgan_dir, "meldataset.py")
if os.path.exists(mel_file):
    with open(mel_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # import 경로 수정
    old_import = 'from env import AttrDict'
    new_import = 'from .env import AttrDict'

    if old_import in content:
        content = content.replace(old_import, new_import)
        with open(mel_file, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"✅ {mel_file} import 경로 수정됨")

# 3. TTS.py에서 BigVGAN 로컬 경로 사용 및 state_dict 로드 수정
tts_file = "/content/GPT-SoVITS/GPT_SoVITS/TTS_infer_pack/TTS.py"
with open(tts_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# vocoder 경로 수정
for i, line in enumerate(lines):
    if 'self.vocoder = BigVGAN.from_pretrained("nvidia/bigvgan_v2_24khz_100band_256x"' in line:
        lines[i] = line.replace(
            '"nvidia/bigvgan_v2_24khz_100band_256x"',
            '"pretrained_models/models--nvidia--bigvgan_v2_24khz_100band_256x"'
        )
        print(f"✅ BigVGAN 경로를 로컬로 변경")
        break

# state_dict 로드 라인 주석 처리
for i, line in enumerate(lines):
    if 'print("loading vocoder", self.vocoder.load_state_dict(state_dict_g))' in line:
        lines[i] = '            # ' + line.lstrip()
        print(f"✅ state_dict 로드 라인 주석 처리")
        break

with open(tts_file, 'w', encoding='utf-8') as f:
    f.writelines(lines)

print("\n✅ 모든 패치 완료!")

🔧 BigVGAN import 경로 수정 중...
✅ state_dict 로드 라인 주석 처리

✅ 모든 패치 완료!


## 🎤 STEP 5: TTS 파이프라인 초기화

## 디버깅 : 환경 확인

In [ ]:
import torch
import numpy as np
import sys

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"NumPy: {np.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch: 2.9.0+cu126
NumPy: 1.26.4
CUDA: True
GPU: Tesla T4


## SciPy 누락 문제 해결

In [ ]:
# 이 코드를 Colab 셀에 복사해서 실행
!pip install scipy

# 설치 확인
import scipy
print(f"✓ SciPy: {scipy.__version__}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 43.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
✓ SciPy: 1.16.3


In [ ]:
"""
🚨 GPT-SoVITS v3 완전 자동 수정 스크립트
BigVGAN 경로 문제를 자동으로 해결합니다.

사용법: 이 전체 코드를 Colab 셀에 복사해서 실행
"""

import os
import re
import sys

def fix_bigvgan_path():
    """TTS.py의 BigVGAN 경로 수정"""
    print("="*60)
    print("🔧 TTS.py BigVGAN 경로 자동 수정")
    print("="*60)

    tts_file = "/content/GPT-SoVITS/GPT_SoVITS/TTS_infer_pack/TTS.py"

    if not os.path.exists(tts_file):
        print(f"❌ 파일을 찾을 수 없습니다: {tts_file}")
        return False

    # 파일 읽기
    with open(tts_file, 'r', encoding='utf-8') as f:
        content = f.read()

    original_content = content
    modified = False

    # 수정 1: now_dir 기반 경로를 절대 경로로
    pattern1 = r'"%s/GPT_SoVITS/pretrained_models/models--nvidia--bigvgan_v2_24khz_100band_256x" % \(now_dir,?\)'
    replacement1 = '"/content/GPT-SoVITS/GPT_SoVITS/pretrained_models/models--nvidia--bigvgan_v2_24khz_100band_256x"'

    if re.search(pattern1, content):
        content = re.sub(pattern1, replacement1, content)
        print("✅ now_dir 기반 경로 → 절대 경로로 수정")
        modified = True

    # 수정 2: 상대 경로를 절대 경로로
    pattern2 = r'"pretrained_models/models--nvidia--bigvgan_v2_24khz_100band_256x"'
    replacement2 = '"/content/GPT-SoVITS/GPT_SoVITS/pretrained_models/models--nvidia--bigvgan_v2_24khz_100band_256x"'

    if re.search(pattern2, content) and replacement2 not in content:
        content = re.sub(pattern2, replacement2, content)
        print("✅ 상대 경로 → 절대 경로로 수정")
        modified = True

    # 수정 3: state_dict 로드 라인 주석 처리
    if 'print("loading vocoder", self.vocoder.load_state_dict(state_dict_g))' in content:
        content = content.replace(
            'print("loading vocoder", self.vocoder.load_state_dict(state_dict_g))',
            '# print("loading vocoder", self.vocoder.load_state_dict(state_dict_g))'
        )
        print("✅ state_dict 로드 라인 주석 처리")
        modified = True

    if modified:
        # 파일 저장
        with open(tts_file, 'w', encoding='utf-8') as f:
            f.write(content)
        print("\n✅ TTS.py 수정 완료!")
        return True
    else:
        print("\nℹ️ 수정할 내용이 없습니다 (이미 수정됨)")
        return True

def verify_bigvgan_model():
    """BigVGAN 모델 파일 존재 확인"""
    print("\n" + "="*60)
    print("📂 BigVGAN 모델 파일 확인")
    print("="*60)

    model_dir = "/content/GPT-SoVITS/GPT_SoVITS/pretrained_models/models--nvidia--bigvgan_v2_24khz_100band_256x"

    if not os.path.exists(model_dir):
        print(f"❌ 디렉토리 없음: {model_dir}")
        print("\n💡 해결 방법:")
        print("!git clone https://huggingface.co/nvidia/bigvgan_v2_24khz_100band_256x \\")
        print(f"    {model_dir}")
        return False

    # 필수 파일 확인
    required_files = [
        "config.json",
        "model.pt"
    ]

    missing_files = []
    for file in required_files:
        filepath = os.path.join(model_dir, file)
        if os.path.exists(filepath):
            print(f"✓ {file}")
        else:
            print(f"✗ {file}")
            missing_files.append(file)

    if missing_files:
        print(f"\n⚠️ {len(missing_files)}개 파일 누락")
        return False
    else:
        print("\n✅ 모든 필수 파일 존재")
        return True

def clear_cache():
    """모듈 캐시 초기화"""
    print("\n" + "="*60)
    print("🧹 모듈 캐시 초기화")
    print("="*60)

    modules_to_clear = [
        'TTS_infer_pack.TTS',
        'TTS_infer_pack',
        'BigVGAN.bigvgan',
        'BigVGAN'
    ]

    for module in modules_to_clear:
        if module in sys.modules:
            del sys.modules[module]
            print(f"✓ {module} 캐시 삭제")

    print("\n✅ 캐시 초기화 완료")

def main():
    """메인 실행 함수"""
    print("\n" + "="*60)
    print("🚀 GPT-SoVITS v3 자동 수정 시작")
    print("="*60 + "\n")

    # 1. BigVGAN 경로 수정
    if not fix_bigvgan_path():
        print("\n❌ 경로 수정 실패")
        return False

    # 2. 모델 파일 확인
    if not verify_bigvgan_model():
        print("\n⚠️ 모델 파일 확인 필요")
        # 모델 파일이 없어도 계속 진행

    # 3. 캐시 초기화
    clear_cache()

    print("\n" + "="*60)
    print("✅ 모든 수정 완료!")
    print("="*60)
    print("\n📋 다음 단계:")
    print("1. TTS 초기화 코드를 다시 실행하세요")
    print("2. 여전히 에러가 발생하면 런타임을 재시작하세요")

    return True

# 스크립트 실행
if __name__ == "__main__":
    success = main()

    if success:
        print("\n💡 TIP: 이제 TTS 초기화 코드를 실행하세요!")
    else:
        print("\n❌ 문제가 발생했습니다. 로그를 확인하세요.")


🚀 GPT-SoVITS v3 자동 수정 시작

🔧 TTS.py BigVGAN 경로 자동 수정
✅ now_dir 기반 경로 → 절대 경로로 수정
✅ state_dict 로드 라인 주석 처리

✅ TTS.py 수정 완료!

📂 BigVGAN 모델 파일 확인
✓ config.json
✗ model.pt

⚠️ 1개 파일 누락

⚠️ 모델 파일 확인 필요

🧹 모듈 캐시 초기화
✓ TTS_infer_pack.TTS 캐시 삭제
✓ TTS_infer_pack 캐시 삭제
✓ BigVGAN.bigvgan 캐시 삭제
✓ BigVGAN 캐시 삭제

✅ 캐시 초기화 완료

✅ 모든 수정 완료!

📋 다음 단계:
1. TTS 초기화 코드를 다시 실행하세요
2. 여전히 에러가 발생하면 런타임을 재시작하세요

💡 TIP: 이제 TTS 초기화 코드를 실행하세요!


In [ ]:
import os
import sys

# Python path 설정
paths_to_add = [
    "/content/GPT-SoVITS",
    "/content/GPT-SoVITS/GPT_SoVITS",
    "/content/GPT-SoVITS/GPT_SoVITS/eres2net"
]

for path in paths_to_add:
    if path not in sys.path:
        sys.path.insert(0, path)

os.chdir("/content/GPT-SoVITS/GPT_SoVITS")

# 모듈 캐시 완전 초기화
for module in list(sys.modules.keys()):
    if 'TTS_infer_pack' in module or 'BigVGAN' in module:
        del sys.modules[module]

from TTS_infer_pack.TTS import TTS, TTS_Config

print("🎙️ v3 TTS 초기화 중...\n")

# v3 모델 설정
tts_config_v3 = TTS_Config()
tts_config_v3.t2s_weights_path = "/content/GPT-SoVITS/GPT_SoVITS/pretrained_models/s1v3.ckpt"
tts_config_v3.vits_weights_path = "/content/GPT-SoVITS/GPT_SoVITS/pretrained_models/s2Gv3.pth"
tts_config_v3.bert_base_path = "/content/GPT-SoVITS/GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large"
tts_config_v3.cnhuhbert_base_path = "/content/GPT-SoVITS/GPT_SoVITS/pretrained_models/chinese-hubert-base"
tts_config_v3.device = "cuda"

tts_pipeline_v3 = TTS(tts_config_v3)

print("✅ v3 TTS 파이프라인 초기화 완료!")

🎙️ v3 TTS 초기화 중...

fall back to default t2s_weights_path: GPT_SoVITS/pretrained_models/gsv-v2final-pretrained/s1bert25hz-5kh-longer-epoch=12-step=369668.ckpt
fall back to default vits_weights_path: GPT_SoVITS/pretrained_models/gsv-v2final-pretrained/s2G2333k.pth
fall back to default bert_base_path: GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large
fall back to default cnhuhbert_base_path: GPT_SoVITS/pretrained_models/chinese-hubert-base
Loading Text2Semantic weights from /content/GPT-SoVITS/GPT_SoVITS/pretrained_models/s1v3.ckpt
Loading VITS weights from /content/GPT-SoVITS/GPT_SoVITS/pretrained_models/s2Gv3.pth. <All keys matched successfully>
Loading BERT weights from /content/GPT-SoVITS/GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large
Loading CNHuBERT weights from /content/GPT-SoVITS/GPT_SoVITS/pretrained_models/chinese-hubert-base
✅ v3 TTS 파이프라인 초기화 완료!


## 📖 STEP 6: JSON 스크립트 기반 오디오북 생성

In [ ]:
# torchcodec 설치
print("📦 torchcodec 설치 중...")
!pip install torchcodec

print("✅ 설치 완료!")

📦 torchcodec 설치 중...
✅ 설치 완료!


## TTS 테스트 : 첫번째 세그먼트만 먼저 재생

In [ ]:
# 🎙️ 전체 동화 오디오북 생성 (v3 한국어)
import json
import soundfile as sf
from pathlib import Path

# JSON 로드
with open("/content/유민이의_특별한_한복_찾기_20251207_2253.json", 'r', encoding='utf-8') as f:
    story_data = json.load(f)

print(f"📖 동화 제목: {story_data['title']}")
print(f"📝 총 {len(story_data['script'])}개 세그먼트\n")

# 출력 디렉토리
output_dir = "/content/audiobook_v3_ko"
Path(output_dir).mkdir(exist_ok=True)

# Narrator 참조
ref_audio = "/content/GPT-SoVITS/sliced/teacher/teacher_0000.wav"
ref_text = "그녀는 남녀를 막론하고 노예들에게 먹을 것과 마실 것을 베풀어 주었고, 그들의 능력에 따라서 이를 분배해 주었습니다."

print("🎬 v3 한국어 오디오북 생성 시작...\n")

for segment in story_data['script']:
    seq = segment['seq']
    text = segment['text']
    audio_filename = segment['audio_file_name']

    print(f"[{seq}/8] 생성 중: {text[:40]}...")

    try:
        inputs = {
            "text": text,
            "text_lang": "ko",
            "ref_audio_path": ref_audio,
            "prompt_text": ref_text,
            "prompt_lang": "ko",
            "streaming_mode": False,
            "return_fragment": False
        }

        result_gen = tts_pipeline_v3.run(inputs)

        for result in result_gen:
            if isinstance(result, tuple) and len(result) == 2:
                sr, audio_data = result

                # 저장
                output_path = f"{output_dir}/{audio_filename}"
                sf.write(output_path, audio_data, sr)

                duration = len(audio_data) / sr
                print(f"  ✅ 완료 ({duration:.1f}초): {audio_filename}\n")
                break

    except Exception as e:
        print(f"  ❌ 오류: {e}\n")

print("="*60)
print("✅ 한국어 오디오북 생성 완료!")
print(f"📁 저장 위치: {output_dir}\n")

# 생성된 파일 확인
!ls -lh {output_dir}

# 첫 번째 파일 재생
from IPython.display import Audio, display
print("\n🎵 첫 번째 세그먼트 재생:")
display(Audio(f"{output_dir}/001_narrator.wav"))

📖 동화 제목: 유민이의 특별한 한복 찾기
📝 총 8개 세그먼트

🎬 v3 한국어 오디오북 생성 시작...

[1/8] 생성 중: 높은 빌딩이 숲을 이루고 우리가 살고 있는 바로 지금, 서울 동대문 근처...
Set seed to 165467130
Parallel Inference Mode Enabled
When parallel inference mode is enabled, SoVITS V3/4 models do not support bucket processing; bucket processing has been automatically disabled.
Actual Input Reference Text: 그녀는 남녀를 막론하고 노예들에게 먹을 것과 마실 것을 베풀어 주었고, 그들의 능력에 따라서 이를 분배해 주었습니다.


[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


mecab installed
############ Segment Text ############
Actual Input Target Text:
높은 빌딩이 숲을 이루고 우리가 살고 있는 바로 지금, 서울 동대문 근처에 사는 유민이는 하루하루가 모험 같았어요. 유민이는 초등학교에 다니는 소녀로, 늘 공주 옷을 입고 싶어 했죠. 어느 날, 유민이는 엄마와 함께 동대문 시장을 방문했어요. 유민이의 엄마는 동대문 시장에서 한복 가게를 운영하는 따뜻하고 자상한 분이었답니다.
Actual Input Target Text (after sentence segmentation):
['높은 빌딩이 숲을 이루고 우리가 살고 있는 바로 지금, 서울 동대문 근처에 사는 유민이는 하루하루가 모험 같았어요. 유민이는 초등학교에 다니는 소녀로, 늘 공주 옷을 입고 싶어 했죠. 어느 날, 유민이는 엄마와 함께 동대문 시장을 방문했어요. 유민이의 엄마는 동대문 시장에서 한복 가게를 운영하는 따뜻하고 자상한 분이었답니다.']
############ Extract Text BERT Features ############


100%|██████████| 1/1 [00:00<00:00,  4.30it/s]


############ 推理 ############
Processed text from the frontend (per sentence): ['높은 빌딩이 숲을 이루고 우리가 살고 있는 바로 지금, 서울 동대문 근처에 사는 유민이는 하루하루가 모험 같았어요. 유민이는 초등학교에 다니는 소녀로, 늘 공주 옷을 입고 싶어 했죠. 어느 날, 유민이는 엄마와 함께 동대문 시장을 방문했어요. 유민이의 엄마는 동대문 시장에서 한복 가게를 운영하는 따뜻하고 자상한 분이었답니다.']
############ Predict Semantic Token ############


...

T2S Decoding EOS [240 -> 678]
############ Synthesize Audio ############
Parallel Synthesis in Progress...
seg_idx: 296
seg_idx: 300
seg_idx: 85
0.000	0.052	6.083	22.835
  ✅ 완료 (17.8초): 007_narrator.wav

[8/8] 생성 중: 유민이의 특별한 한복 찾기는 이렇게 끝이 났답니다. 그러나 유민이의 마음...
Set seed to 1546388439
Parallel Inference Mode Enabled
When parallel inference mode is enabled, SoVITS V3/4 models do not support bucket processing; bucket processing has been automatically disabled.
Actual Input Reference Text: 그녀는 남녀를 막론하고 노예들에게 먹을 것과 마실 것을 베풀어 주었고, 그들의 능력에 따라서 이를 분배해 주었습니다.
############ Segment Text ############
Actual Input Target Text:
유민이의 특별한 한복 찾기는 이렇게 끝이 났답니다. 그러나 유민이의 마음속에는 언제나 전통과 문화의 중요함이 자리 잡고 있었어요. 그리고 그 덕분에 유민이는 더욱 멋진 어른으로 자라날 수 있었답니다.
Actual Input Target Text (after sentence segmentation):
['유민이의 특별한 한복 찾기는 이렇게 끝이 났답니다. 그러나 유민이의 마음속에는 언제나 전통과 문화의 중요함이 자리 잡고 있었어요. 그리고 그 덕분에 유민이는 더욱 멋진 어른으로 자라날 수 있었답니다.']
############ Extract Text BERT Features ############


100%|██████████| 1/1 [00:00<00:00, 27.08it/s]


############ 推理 ############
Processed text from the frontend (per sentence): ['유민이의 특별한 한복 찾기는 이렇게 끝이 났답니다. 그러나 유민이의 마음속에는 언제나 전통과 문화의 중요함이 자리 잡고 있었어요. 그리고 그 덕분에 유민이는 더욱 멋진 어른으로 자라날 수 있었답니다.']
############ Predict Semantic Token ############


 23%|██▎       | 343/1500 [00:05<00:17, 66.61it/s]


T2S Decoding EOS [240 -> 584]
############ Synthesize Audio ############
Parallel Synthesis in Progress...
seg_idx: 33
seg_idx: 254
0.000	0.041	5.156	17.212
  ✅ 완료 (14.1초): 008_narrator.wav

✅ 한국어 오디오북 생성 완료!
📁 저장 위치: /content/audiobook_v3_ko

total 6.4M
-rw-r--r-- 1 root root 1.2M Dec 11 10:59 001_narrator.wav
-rw-r--r-- 1 root root 634K Dec 11 11:00 002_narrator.wav
-rw-r--r-- 1 root root 971K Dec 11 11:00 003_narrator.wav
-rw-r--r-- 1 root root 852K Dec 11 11:01 004_narrator.wav
-rw-r--r-- 1 root root 626K Dec 11 11:01 005_narrator.wav
-rw-r--r-- 1 root root 809K Dec 11 11:01 006_narrator.wav
-rw-r--r-- 1 root root 837K Dec 11 11:02 007_narrator.wav
-rw-r--r-- 1 root root 662K Dec 11 11:02 008_narrator.wav

🎵 첫 번째 세그먼트 재생:


In [ ]:
# 🎙️ 전체 동화 오디오북 생성 (v3 한국어) - 한국 시간 적용
import json
import soundfile as sf
from pathlib import Path
from datetime import datetime, timedelta, timezone
import os

# ============================================================
# 한국 시간대 설정
# ============================================================

def get_kst_now():
    """한국 표준시(KST, UTC+9) 현재 시간 반환"""
    return datetime.now(timezone(timedelta(hours=9)))

# ============================================================
# JSON 로드
# ============================================================

json_file = "/content/유민이의_특별한_한복_찾기_20251207_2253.json"

with open(json_file, 'r', encoding='utf-8') as f:
    story_data = json.load(f)

print(f"📖 동화 제목: {story_data['title']}")
print(f"📝 총 {len(story_data['script'])}개 세그먼트\n")

# ============================================================
# 출력 디렉토리 및 파일명 생성
# ============================================================

# 사용된 캐릭터 수집
used_characters = set()
for segment in story_data['script']:
    character = segment.get('character', 'narrator')
    used_characters.add(character)

# 캐릭터 리스트를 정렬된 문자열로 변환
characters_str = "_".join(sorted(used_characters))

# 현재 날짜/시간 (한국 시간, UTC+9)
now_kst = get_kst_now()
timestamp = now_kst.strftime("%Y%m%d_%H%M")

# 동화 제목 (파일명으로 사용 가능하게 정리)
story_title = story_data['title'].replace(" ", "_").replace("/", "_")

# 출력 디렉토리 이름 생성
output_dir_name = f"{story_title}_{characters_str}_{timestamp}"
output_dir = f"/content/audiobook_outputs/{output_dir_name}"

# 디렉토리 생성
Path(output_dir).mkdir(parents=True, exist_ok=True)

print(f"📁 출력 디렉토리: {output_dir}")
print(f"🎭 사용된 캐릭터: {', '.join(sorted(used_characters))}")
print(f"📅 생성 일시: {now_kst.strftime('%Y년 %m월 %d일 %H시 %M분')} (KST)")
print(f"🌏 시간대: 한국 표준시 (UTC+9)\n")

# ============================================================
# 참조 오디오 설정
# ============================================================

# Narrator 참조 (기본값)
ref_audio = "/content/GPT-SoVITS/sliced/teacher/teacher_0000.wav"
ref_text = "그녀는 남녀를 막론하고 노예들에게 먹을 것과 마실 것을 베풀어 주었고, 그들의 능력에 따라서 이를 분배해 주었습니다."

# 참조 오디오 파일 존재 확인
if not os.path.exists(ref_audio):
    print("⚠️ 참조 오디오 파일이 없습니다!")
    print(f"경로: {ref_audio}")
    print("\n해결 방법:")
    print("1. STEP 0.5 (데이터 준비)를 실행하세요")
    print("2. 또는 아래 코드로 임시 참조 오디오를 생성하세요:\n")
    print("import numpy as np")
    print("from pathlib import Path")
    print("temp_dir = Path('/content/GPT-SoVITS/sliced/teacher')")
    print("temp_dir.mkdir(parents=True, exist_ok=True)")
    print("audio = np.zeros(32000, dtype=np.float32)")
    print("sf.write(str(temp_dir / 'teacher_0000.wav'), audio, 32000)")
    raise FileNotFoundError(f"참조 오디오 파일 없음: {ref_audio}")

print(f"✅ 참조 오디오: {ref_audio}")
print(f"✅ 참조 텍스트: {ref_text}\n")

# ============================================================
# 오디오북 생성
# ============================================================

print("🎬 오디오북 생성 시작...\n")
print("="*60)

generated_files = []
failed_segments = []
total_duration = 0

for segment in story_data['script']:
    seq = segment['seq']
    text = segment['text']
    audio_filename = segment['audio_file_name']
    character = segment.get('character', 'narrator')

    # 진행 상황 표시
    print(f"\n[{seq}/{len(story_data['script'])}] {character}")
    print(f"📝 {text[:60]}{'...' if len(text) > 60 else ''}")

    try:
        inputs = {
            "text": text,
            "text_lang": "ko",
            "ref_audio_path": ref_audio,
            "prompt_text": ref_text,
            "prompt_lang": "ko",
            "streaming_mode": False,
            "return_fragment": False
        }

        result_gen = tts_pipeline_v3.run(inputs)

        for result in result_gen:
            if isinstance(result, tuple) and len(result) == 2:
                sr, audio_data = result

                # 저장
                output_path = f"{output_dir}/{audio_filename}"
                sf.write(output_path, audio_data, sr)

                duration = len(audio_data) / sr
                total_duration += duration

                generated_files.append({
                    'seq': seq,
                    'filename': audio_filename,
                    'path': output_path,
                    'duration': duration,
                    'character': character
                })

                print(f"✅ 생성 완료 ({duration:.1f}초)")
                break

    except Exception as e:
        error_msg = str(e)
        print(f"❌ 오류: {error_msg[:100]}")
        failed_segments.append({
            'seq': seq,
            'filename': audio_filename,
            'error': error_msg
        })

# ============================================================
# 결과 요약
# ============================================================

print("\n" + "="*60)
print("📊 생성 결과 요약")
print("="*60)

print(f"\n✅ 성공: {len(generated_files)}개 / {len(story_data['script'])}개")
print(f"❌ 실패: {len(failed_segments)}개")
print(f"⏱️  총 오디오 길이: {total_duration:.1f}초 ({total_duration/60:.1f}분)")
print(f"📁 저장 위치: {output_dir}")

# 생성된 파일 목록
if generated_files:
    print(f"\n📄 생성된 파일:")
    for item in generated_files:
        print(f"  {item['seq']:2d}. {item['filename']:20s} ({item['duration']:5.1f}초) [{item['character']}]")

# 실패한 세그먼트
if failed_segments:
    print(f"\n⚠️ 실패한 세그먼트:")
    for item in failed_segments:
        print(f"  {item['seq']:2d}. {item['filename']:20s}")
        print(f"      에러: {item['error'][:80]}")

# ============================================================
# 메타데이터 저장
# ============================================================

metadata = {
    "title": story_data['title'],
    "generated_at": now_kst.isoformat(),
    "generated_at_kst": now_kst.strftime('%Y-%m-%d %H:%M:%S KST'),
    "timezone": "Asia/Seoul (UTC+9)",
    "total_segments": len(story_data['script']),
    "successful_segments": len(generated_files),
    "failed_segments": len(failed_segments),
    "total_duration_seconds": total_duration,
    "used_characters": sorted(list(used_characters)),
    "output_directory": output_dir,
    "files": generated_files,
    "failures": failed_segments
}

metadata_file = f"{output_dir}/metadata.json"
with open(metadata_file, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print(f"\n💾 메타데이터 저장: metadata.json")

# ============================================================
# 파일 목록 출력
# ============================================================

print(f"\n📂 생성된 파일 목록:")
print("="*60)
!ls -lh {output_dir}

# ============================================================
# 첫 번째 오디오 재생
# ============================================================

if generated_files:
    from IPython.display import Audio, display

    first_file = generated_files[0]['path']
    print(f"\n🎵 첫 번째 세그먼트 재생:")
    print(f"   {generated_files[0]['filename']}")
    display(Audio(first_file))

    # ============================================================
    # 전체 오디오 병합 (선택사항)
    # ============================================================

    print("\n💡 TIP: 전체 오디오를 하나로 병합하려면 다음 코드를 실행하세요:")
    print("-"*60)
    print(f"""
import soundfile as sf
import numpy as np

# 모든 오디오 파일 로드
audio_segments = []
sample_rate = None

for item in sorted({generated_files}, key=lambda x: x['seq']):
    audio, sr = sf.read(item['path'])
    if sample_rate is None:
        sample_rate = sr
    audio_segments.append(audio)

# 병합 (0.5초 무음 간격 추가)
silence = np.zeros(int(sample_rate * 0.5))
merged_audio = []
for audio in audio_segments:
    merged_audio.append(audio)
    merged_audio.append(silence)

merged_audio = np.concatenate(merged_audio[:-1])  # 마지막 무음 제거

# 저장
merged_path = "{output_dir}/FULL_{story_title}.wav"
sf.write(merged_path, merged_audio, sample_rate)

print(f"✅ 전체 오디오 병합 완료: {{merged_path}}")
print(f"   전체 길이: {{len(merged_audio)/sample_rate:.1f}}초 ({{len(merged_audio)/sample_rate/60:.1f}}분)")
display(Audio(merged_path))
""")
    print("-"*60)

    # ============================================================
    # 구글 드라이브 저장 안내
    # ============================================================

    print(f"\n💾 구글 드라이브에 저장하려면:")
    print("-"*60)
    print(f"""
from google.colab import drive
import shutil

# 구글 드라이브 마운트 (아직 안 했다면)
drive.mount('/content/drive')

# 저장 경로
drive_path = "/content/drive/MyDrive/P_dev/audiobook_outputs/{output_dir_name}"

# 복사
shutil.copytree("{output_dir}", drive_path, dirs_exist_ok=True)

print(f"✅ 구글 드라이브에 저장 완료: {{drive_path}}")
""")
    print("-"*60)

    # ============================================================
    # ZIP 다운로드 안내
    # ============================================================

    print(f"\n📥 ZIP 파일로 다운로드하려면:")
    print("-"*60)
    print(f"""
!cd /content/audiobook_outputs && zip -r {output_dir_name}.zip {output_dir_name}

from google.colab import files
files.download('/content/audiobook_outputs/{output_dir_name}.zip')
""")
    print("-"*60)

else:
    print("\n❌ 생성된 오디오 파일이 없습니다.")
    print("\n문제 해결:")
    print("1. 참조 오디오 파일 경로 확인")
    print("2. TTS 파이프라인 초기화 확인")
    print("3. 에러 메시지 확인")

print("\n" + "="*60)
print(f"✅ 오디오북 생성 완료! ({now_kst.strftime('%H:%M:%S KST')})")
print("="*60)

📖 동화 제목: 유민이의 특별한 한복 찾기
📝 총 8개 세그먼트

📁 출력 디렉토리: /content/audiobook_outputs/유민이의_특별한_한복_찾기_narrator_20251211_2011
🎭 사용된 캐릭터: narrator
📅 생성 일시: 2025년 12월 11일 20시 11분 (KST)
🌏 시간대: 한국 표준시 (UTC+9)

✅ 참조 오디오: /content/GPT-SoVITS/sliced/teacher/teacher_0000.wav
✅ 참조 텍스트: 그녀는 남녀를 막론하고 노예들에게 먹을 것과 마실 것을 베풀어 주었고, 그들의 능력에 따라서 이를 분배해 주었습니다.

🎬 오디오북 생성 시작...


[1/8] narrator
📝 높은 빌딩이 숲을 이루고 우리가 살고 있는 바로 지금, 서울 동대문 근처에 사는 유민이는 하루하루가 모험 같...
Set seed to 3612805378
Parallel Inference Mode Enabled
When parallel inference mode is enabled, SoVITS V3/4 models do not support bucket processing; bucket processing has been automatically disabled.
Actual Input Reference Text: 그녀는 남녀를 막론하고 노예들에게 먹을 것과 마실 것을 베풀어 주었고, 그들의 능력에 따라서 이를 분배해 주었습니다.
############ Segment Text ############
Actual Input Target Text:
높은 빌딩이 숲을 이루고 우리가 살고 있는 바로 지금, 서울 동대문 근처에 사는 유민이는 하루하루가 모험 같았어요. 유민이는 초등학교에 다니는 소녀로, 늘 공주 옷을 입고 싶어 했죠. 어느 날, 유민이는 엄마와 함께 동대문 시장을 방문했어요. 유민이의 엄마는 동대문 시장에서 한복 가게를 운영하는 따뜻하고 자상한 분이었답니다.
Actual Input Target Text (

100%|██████████| 1/1 [00:00<00:00, 27.73it/s]


############ 推理 ############
Processed text from the frontend (per sentence): ['높은 빌딩이 숲을 이루고 우리가 살고 있는 바로 지금, 서울 동대문 근처에 사는 유민이는 하루하루가 모험 같았어요. 유민이는 초등학교에 다니는 소녀로, 늘 공주 옷을 입고 싶어 했죠. 어느 날, 유민이는 엄마와 함께 동대문 시장을 방문했어요. 유민이의 엄마는 동대문 시장에서 한복 가게를 운영하는 따뜻하고 자상한 분이었답니다.']
############ Predict Semantic Token ############


 40%|███▉      | 596/1500 [00:07<00:12, 75.01it/s]


T2S Decoding EOS [240 -> 837]
############ Synthesize Audio ############
Parallel Synthesis in Progress...
seg_idx: 105
seg_idx: 15
seg_idx: 178
seg_idx: 38
0.000	0.041	7.954	27.935
✅ 생성 완료 (24.2초)

[2/8] narrator
📝 "엄마, 오늘은 특별한 한복을 찾고 싶어요!" 유민이는 엄마에게 말했어요. 엄마는 미소를 지으며 "그래, 유...
Set seed to 4093444261
Parallel Inference Mode Enabled
When parallel inference mode is enabled, SoVITS V3/4 models do not support bucket processing; bucket processing has been automatically disabled.
Actual Input Reference Text: 그녀는 남녀를 막론하고 노예들에게 먹을 것과 마실 것을 베풀어 주었고, 그들의 능력에 따라서 이를 분배해 주었습니다.
############ Segment Text ############
Actual Input Target Text:
。"엄마, 오늘은 특별한 한복을 찾고 싶어요!" 유민이는 엄마에게 말했어요. 엄마는 미소를 지으며 "그래, 유민아. 오늘은 정말 특별한 날이 될 거야."라고 대답했지요.
Actual Input Target Text (after sentence segmentation):
['。"엄마, 오늘은 특별한 한복을 찾고 싶어요!" 유민이는 엄마에게 말했어요. 엄마는 미소를 지으며 "그래, 유민아. 오늘은 정말 특별한 날이 될 거야."라고 대답했지요.']
############ Extract Text BERT Features ############


...

100%|██████████| 1/1 [00:00<00:00, 29.99it/s]


############ 推理 ############
Processed text from the frontend (per sentence): ['유민이의 특별한 한복 찾기는 이렇게 끝이 났답니다. 그러나 유민이의 마음속에는 언제나 전통과 문화의 중요함이 자리 잡고 있었어요. 그리고 그 덕분에 유민이는 더욱 멋진 어른으로 자라날 수 있었답니다.']
############ Predict Semantic Token ############


 24%|██▍       | 358/1500 [00:04<00:15, 74.20it/s]


T2S Decoding EOS [240 -> 599]
############ Synthesize Audio ############
Parallel Synthesis in Progress...
seg_idx: 30
seg_idx: 256
0.000	0.037	4.831	17.233
✅ 생성 완료 (14.7초)

📊 생성 결과 요약

✅ 성공: 8개 / 8개
❌ 실패: 0개
⏱️  총 오디오 길이: 140.4초 (2.3분)
📁 저장 위치: /content/audiobook_outputs/유민이의_특별한_한복_찾기_narrator_20251211_2011

📄 생성된 파일:
   1. 001_narrator.wav     ( 24.2초) [narrator]
   2. 002_narrator.wav     ( 14.3초) [narrator]
   3. 003_narrator.wav     ( 18.7초) [narrator]
   4. 004_narrator.wav     ( 18.4초) [narrator]
   5. 005_narrator.wav     ( 13.6초) [narrator]
   6. 006_narrator.wav     ( 18.0초) [narrator]
   7. 007_narrator.wav     ( 18.5초) [narrator]
   8. 008_narrator.wav     ( 14.7초) [narrator]

💾 메타데이터 저장: metadata.json

📂 생성된 파일 목록:
total 6.5M
-rw-r--r-- 1 root root 1.2M Dec 11 11:12 001_narrator.wav
-rw-r--r-- 1 root root 669K Dec 11 11:12 002_narrator.wav
-rw-r--r-- 1 root root 876K Dec 11 11:13 003_narrator.wav
-rw-r--r-- 1 root root 864K Dec 11 11:13 004_narrator.wav
-rw-r--r-- 1 root 


💡 TIP: 전체 오디오를 하나로 병합하려면 다음 코드를 실행하세요:
------------------------------------------------------------

import soundfile as sf
import numpy as np

# 모든 오디오 파일 로드
audio_segments = []
sample_rate = None

for item in sorted([{'seq': 1, 'filename': '001_narrator.wav', 'path': '/content/audiobook_outputs/유민이의_특별한_한복_찾기_narrator_20251211_2011/001_narrator.wav', 'duration': 24.226, 'character': 'narrator'}, {'seq': 2, 'filename': '002_narrator.wav', 'path': '/content/audiobook_outputs/유민이의_특별한_한복_찾기_narrator_20251211_2011/002_narrator.wav', 'duration': 14.265708333333333, 'character': 'narrator'}, {'seq': 3, 'filename': '003_narrator.wav', 'path': '/content/audiobook_outputs/유민이의_특별한_한복_찾기_narrator_20251211_2011/003_narrator.wav', 'duration': 18.685458333333333, 'character': 'narrator'}, {'seq': 4, 'filename': '004_narrator.wav', 'path': '/content/audiobook_outputs/유민이의_특별한_한복_찾기_narrator_20251211_2011/004_narrator.wav', 'duration': 18.417208333333335, 'character': 'narrator'}, {'seq': 5, 'file